# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Defaulting to user installation because normal site-packages is not writeable


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [1]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [2]:
# 物理，数学，生物，计算机，汉语言文学

physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

physics_template = """你是一位非常聪明的生物学家。
你擅长以简洁易懂的方式回答关于生物的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

math_template = """你是一位很棒的计算机科学家。你擅长回答计算机问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

physics_template = """你是一位非常聪明的汉语言文学教授。
你擅长以简洁易懂的方式回答关于汉语言文学的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

In [3]:
# 物理，数学，生物，计算机，汉语言文学
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": physics_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": math_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": physics_template,
    },
]

In [4]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [5]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

C:\Scoop\apps\miniconda3\current\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [6]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [7]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [8]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [9]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语言文学: 适用于回答汉语言文学问题']


In [10]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题


In [11]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [12]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题

<< INPUT >>
{input}

<

In [13]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [14]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.
{'input': '黑体辐射是什么？?', 'text': '\n\n黑体辐射是指一个物体在不同温度下所发出的热辐射，其能量和波长分布由物体的温度决定。通常，在较低温度下，物体会发出长波长的红外辐射；而在较高温度下，物体则会发出短波长的紫外辐射。黑体辐射是研究热力学和量子物理学中重要的现象之一。'}


In [15]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '大于40的第一个质数是多少'}
> Finished chain.
{'input': '大于40的第一个质数是多少', 'text': '？\n\n首先，我们需要定义什么是质数。质数是指只能被1和自身整除的正整数。因此，我们需要从41开始，依次检查每一个数是否为质数。\n\n首先，我们可以定义一个函数is_prime来判断一个数是否为质数。该函数接受一个参数n，如果n能被2到n-1之间的任何数整除，就返回False，否则返回True。\n\n接下来，我们可以使用一个循环来从41开始依次检查每一个数是否为质数。当我们找到第一个质数后，就可以将其打印出来并结束循环。\n\n以下是一个可能的实现：\n\n```python\ndef is_prime(n):\n    for i in range(2, n):\n        if n % i == 0:\n            return False\n    return True\n\nfor num in range(41, 100):\n    if is_prime(num):\n        print("大于40的第一个质数是：%d" % num)\n        break\n```\n\n输出结果'}


In [16]:
router_chain.verbose = True

In [18]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '什么是黑洞？'}
> Finished chain.
{'input': '什么是黑洞？', 'text': '\n\n黑洞是宇宙中一种极为密集的天体，它的引力非常强大，甚至连光也无法逃脱它的吸引力。它的形成是由于恒星在死亡过程中，其内部的核燃料耗尽，无法抵抗自身的引力而坍缩形成的。由于黑洞吸收了大量的物质和能量，它们在宇宙中起着重要的作用，但目前科学家们仍在探索黑洞的性质和特点。'}


In [19]:
router_chain.verbose = False

In [21]:
print(chain.invoke("动物都有哪些类别？"))



> Entering new MultiPromptChain chain...
None: {'input': '动物都有哪些类别？'}
> Finished chain.
{'input': '动物都有哪些类别？', 'history': '', 'text': ' 哦，这是一个很有趣的问题！据我所知，动物可以分为很多不同的类别，例如哺乳动物、鸟类、爬行动物、两栖动物、鱼类等等。每个类别都有自己独特的特征和生活方式。还有一些更细分的类别，比如猫科动物、犬科动物、啮齿动物等等。总的来说，动物的分类是根据它们的形态、生理特征和进化关系来确定的。'}


In [20]:
print(chain.invoke("世界上最好的计算机语言是什么？"))



> Entering new MultiPromptChain chain...
计算机: {'input': '世界上最好的计算机语言是什么？'}
> Finished chain.
{'input': '世界上最好的计算机语言是什么？', 'text': '\n这是一个很有意思的问题，但是很难给出一个确切的答案。因为每种计算机语言都有其独特的优点和适用的场合。\n\n总的来说，计算机语言可以分为高级语言和低级语言。高级语言更加易于理解和学习，比如Java、Python和C++。它们提供了丰富的库和框架，可以快速地构建复杂的应用程序。而低级语言（如汇编语言）更加接近计算机底层，可以直接控制硬件，但是学习和使用难度较大。\n\n另外，计算机语言也可以根据用途进行分类，比如Web开发常用的语言有HTML、CSS和JavaScript，数据科学领域常用的语言有R和Python，游戏开发领域常用的语言有'}


In [22]:
print(chain.invoke("中国古代文学有哪些特点？"))



> Entering new MultiPromptChain chain...
汉语言文学: {'input': '中国古代文学有哪些特点？'}
> Finished chain.
{'input': '中国古代文学有哪些特点？', 'text': '\n\n中国古代文学有以下几个主要特点：\n1. 唯美性：中国古代文学以唯美的表现手法来描绘自然和人物，追求意境和情感的表达。\n\n2. 思想性：中国古代文学作品往往蕴含着深刻的思想，反映着中国古代社会的价值观和精神追求。\n\n3. 历史性：中国古代文学作品往往反映了当时社会的历史和文化背景，具有重要的历史意义。\n\n4. 符号性：中国古代文学作品运用丰富的象征和隐喻手法，通过寓意和比喻来表达思想和情感。\n\n5. 叙事性：中国古代文学作品多以叙事的方式来呈现，通过生动'}


In [23]:
print(chain.invoke("叶绿体是什么？"))



> Entering new MultiPromptChain chain...
生物: {'input': '叶绿体是什么？'}
> Finished chain.
{'input': '叶绿体是什么？', 'text': '\n\n叶绿体是植物细胞中的一种细胞器，它主要负责光合作用，将光能转化为化学能，为植物提供能量。它具有独特的绿色色素，可以吸收光能，并将其转化为化学能。同时，叶绿体也参与到植物的许多其他生命活动中。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains